In [1]:
import pymysql

이 코드를 실행하기 전에 터미널을 열어서 mariadb 를 실행시키는 것이 필요하다. 

In [2]:
connection = pymysql.connect(host='localhost', port=3306, db='Investar',
                            user='root', passwd='Rktnek0806^^', autocommit=True)
cursor = connection.cursor()
cursor.execute("SELECT VERSION();")
result = cursor.fetchone()

print("MariaDB version : {}".format(result))

connection.close()


MariaDB version : ('10.6.4-MariaDB',)


In [3]:
class DBUpdater:
    
    def __init__(self):
        """ 생성자: MariaDB 연결 및 종목코드 딕셔너리 생성 """

        self.conn = pymysql.connect(host='localhost', user='root',
                                    password='Rktnek0806^^', db='Investar', charset='utf8')

        with self.conn.cursor() as curs:
            sql = """

            CREATE TABLE IF NOT EXIST company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))

            """
            curs.execute(sql)
            
            sql = """

            CREATE TABLE IF NOT EXIST daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))

            """

            curs.execute(sql)

        self.conn.commit()
        self.codes = dict()
        self.update_comp_info()

    def __del__(self):
        """소멸자: MariaDB 연결 해제 """
        self.conn.close()

### 5.3.5 종목코드 구하기 

In [4]:
import pandas as pd 


In [5]:
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
krx = pd.read_html(url, header=0)[0]
    

In [6]:
krx

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시
1,DRB동일,4840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
3,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
4,KPX케미칼,25000,기초 화학물질 제조업,"PPG,PU RESIN,우레탄수지,대향막박리재,반도체 CM PAD,Polyether...",1994-12-27,12월,"양준영, 최재호, 이찬수(각자대표이사)",http://www.kpxchemical.com,서울특별시
...,...,...,...,...,...,...,...,...,...
2497,토마토시스템,393210,소프트웨어 개발 및 공급업,UI솔루션,2021-10-14,12월,이상돈,http://www.tomatosystem.co.kr/,서울특별시
2498,틸론,217880,소프트웨어 개발 및 공급업,"Cloud Solution (D,A,E,Rstation)",2015-05-28,12월,최백준,http://www.tilon.co.kr,서울특별시
2499,플럼라인생명과학,222670,자연과학 및 공학 연구개발업,"강아지암치료제,돼지DNA테라피",2015-07-28,12월,김앤토니경태,http://www.plsdna.com,서울특별시
2500,한국미라클피플사,331660,기타 화학제품 제조업,"은나노스텝, 닥터오렌지 등",2019-10-28,12월,이호경,http://www.kmpc.co.kr,경기도


In [7]:
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
krx = pd.read_html(url, header=0)[0]
krx = krx[['종목코드', '회사명']]
krx = krx.rename(columns={'종목코드':'code', '회사명':'company'})
krx.code = krx.code.map('{:06d}'.format)
                        

In [8]:
krx

,code,company
0,000210,DL
1,004840,DRB동일
2,155660,DSR
3,078930,GS
4,025000,KPX케미칼
...,...,...
2497,393210,토마토시스템
2498,217880,틸론
2499,222670,플럼라인생명과학
2500,331660,한국미라클피플사
